In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
from tensorflow import keras
tf.load_library("/etc/alternatives/libcudnn_so")
import matplotlib.pyplot as plt

import os
import numpy as np
import utils
import ner_module
from ner_module import *

2023-11-05 23:43:48.699043: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-05 23:43:48.699068: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-05 23:43:48.699084: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


The NER model module importing from directory  /media/daniel/HDD1/AI574/Project


/home/daniel/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cluster_spec = {
    "worker": ["worker1:port", "worker2:port"],
    "chief": ["chief1:port"]
}
cluster_resolver = tf.distribute.cluster_resolver.TFConfigClusterResolver(cluster_spec)
strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy(
    communication=tf.distribute.experimental.CollectiveCommunication.AUTO,
    cluster_resolver=cluster_resolver
)
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

Number of devices: 2


In [3]:
ner_data_importer = ImportData(source="existing")
train_dataset, val_dataset, lookup_layer = ner_data_importer.import_data()

Importing preloaded training and validation data
File /media/daniel/HDD1/AI574/Project/vocabulary.txt exists. Overwriting existing file.


In [14]:
train = "N"
if train == "Y":
    train_model = TrainModel()
    train_model.build_model()
    train_model.compile_model()
    history = train_model.train(train_data=train_dataset,
                                val_data = val_dataset,
                                epochs=500, batch_size=32)
    train_model.save_model('NER_saved_recent')

    plt.figure(figsize=(14, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')

    plt.tight_layout()
    plt.show()

In [5]:
ner_model = tf.keras.models.load_model('NER_saved_recent', custom_objects={'CustomNonPaddingTokenLoss': CustomNonPaddingTokenLoss})

In [6]:
def lowercase_and_convert_to_ids(tokens):
    tokens = tf.strings.lower(tokens)
    return lookup_layer(tokens)
def tokenize_and_convert_to_ids(text):
    tokens = text.split()
    return lowercase_and_convert_to_ids(tokens)

sample_input = tokenize_and_convert_to_ids(
    "eu rejects german call to boycott british lamb from Steve parson the funky Parson"
)
sample_input = tf.reshape(sample_input, shape=[1, -1])
print(sample_input)

output = ner_model.predict(sample_input)
prediction = np.argmax(output, axis=-1)[0]
prediction = [MAPPING[i] for i in prediction]
print(prediction)

tf.Tensor(
[[  988 10950   204   628     6  3938   215  5773    26  1036     0     1
      0     0]], shape=(1, 14), dtype=int64)
1/1 [==============================] - 1s 1s/step
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O']


In [8]:
data_dir = '/media/daniel/HDD1/AI574/gutenberg/data/raw'
selected_works_path = '/media/daniel/HDD1/AI574/Project/selected_works_idx.csv'
import csv
import re
cur_txt = ''
with open(selected_works_path, mode='r', newline='') as file:
    csv_reader = csv.reader(file)
    i = 0
    for row in csv_reader:
        if i == 3:
            break
        target_path = os.path.join('/media/daniel/HDD1/AI574/gutenberg/data/raw',row[0]+'_raw.txt')
        print(target_path)
        rows_as_strings = []
        with open(target_path, mode='r', newline='') as file:
            csv_reader = csv.reader(file)
            for row in csv_reader:
                row_string = ' '.join(row)
                rows_as_strings.append(row_string)
        cur_txt = '\n'.join(rows_as_strings)                
        cur_txt = utils.remove_special_characters(cur_txt, remove_digits=True).lower()

        # print(len(cur_txt))
        i += 1

/media/daniel/HDD1/AI574/gutenberg/data/raw/PG10002_raw.txt
/media/daniel/HDD1/AI574/gutenberg/data/raw/PG10005_raw.txt
/media/daniel/HDD1/AI574/gutenberg/data/raw/PG10006_raw.txt


In [9]:
cur_txt = cur_txt[:20000]
orig_words = cur_txt.split()
# print(cur_txt)

In [10]:
sample_input = tokenize_and_convert_to_ids(cur_txt)
sample_input = tf.reshape(sample_input, shape=[1, -1])
# print(sample_input)

output = ner_model.predict(sample_input)
prediction = np.argmax(output, axis=-1)[0]
prediction = [MAPPING[i] for i in prediction]
# print(prediction)

1/1 [==============================] - 0s 97ms/step


In [13]:
positions = [index for index, value in enumerate(prediction) if value != 'O']
NER_words = list(set([orig_words[index] for index in positions]))
NER_words


['knight', 'john', 'gore', 'james', 'god']

In [12]:
utils.named_persons_w_spacy(cur_txt)

['robert',
 'garvindavemorgan',
 'gore',
 'giovanni',
 'joanna',
 'johnaddington',
 'james',
 'trojan']